# setup

In [1]:
import torch
import os

In [2]:
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

from configs.baseline import LoadDataConfig, Downstream_cnn_args
# from configs.gate import LoadDataConfig, Downstream_cnn_args
# from configs.fake import LoadDataConfig, Downstream_cnn_args
from data.load_data import LoadData
from models.baseline import ResnetBaseline
from configs.moe import MoE_cnn_args
from models.moe import ResnetMoE
from utils import synthesis

# init

In [3]:
# model_label = 'gate'
model_label = 'moe'

In [4]:
loader_config = LoadDataConfig()
resnet_config = Downstream_cnn_args()
moe_config = MoE_cnn_args()

In [5]:
dataloader = LoadData(**loader_config.__dict__)
if model_label == 'gate':
    reference = ResnetBaseline(**resnet_config.__dict__)
    from configs.gate import LoadDataConfig, Downstream_cnn_args
if model_label == 'moe':
    reference = ResnetMoE(**moe_config.__dict__)
    from configs.baseline import LoadDataConfig, Downstream_cnn_args

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:

moe_config = MoE_cnn_args()

# threshould

In [6]:
model = torch.load('output/{}.pt'.format(model_label))
assert model.state_dict().keys() == reference.state_dict().keys()
model = model.to(device)

In [7]:
val_dl = dataloader.get_val_dataloader()
best_f1s, best_thresholds = synthesis(model, val_dl, None, device)
best_f1s, best_thresholds

  0%|          | 0/272 [00:00<?, ?it/s]

100%|██████████| 272/272 [06:34<00:00,  1.45s/it]


([0.03, 0.01, 0.0],
 [0.33697031210352707, 0.1683190129984578, 0.9424144275656523])

# test

In [8]:
test_dl = dataloader.get_test_dataloader()
all_binary_results, all_true_labels, metrics_dict = synthesis(model, test_dl, best_thresholds, device)
metrics_dict

100%|██████████| 135/135 [03:36<00:00,  1.61s/it]


IndexError: index 3 is out of bounds for dimension 1 with size 3